In [99]:
import os
raw_audio = dict()


directory = 'D:/mainproject/babycrydata/hungry'
for filename in os.listdir(directory):
    if filename.endswith(".wav"): 
        raw_audio[os.path.join(directory, filename)] = 'hungry'
    else:
        continue

directory = 'D:/mainproject/babycrydata/pain'
for filename in os.listdir(directory):
    if filename.endswith(".wav"): 
        raw_audio[os.path.join(directory, filename)] = 'pain'
    else:
        continue
        
directory = 'D:/mainproject/babycrydata/discomfort'
for filename in os.listdir(directory):
    if filename.endswith(".wav"): 
        raw_audio[os.path.join(directory, filename)] = 'discomfort'
    else:
        continue

directory = 'D:/mainproject/babycrydata/tired'
for filename in os.listdir(directory):
    if filename.endswith(".wav"): 
        raw_audio[os.path.join(directory, filename)] = 'tired'
    else:
        continue


#print (raw_audio)

import wave 
import math
from pydub import AudioSegment
from pydub.utils import make_chunks

def chop_song(filename, folder):
    #print(filename)
    head, tail = os.path.split(filename)
    shortfilename = tail
    #print(shortfilename)
    myaudio = AudioSegment.from_file(filename, "wav") 
    chunk_length_ms = 1000 # pydub calculates in millisec
    chunks = make_chunks(myaudio, chunk_length_ms) #Make chunks of one sec
    #Export all of the individual chunks as wav files

    for i, chunk in enumerate(chunks):
        chunk_name = folder + shortfilename + 'snippet' + str(i+1) + '.wav'
        print ("exporting", chunk_name)
        chunk.export( 'D:/mainproject/babycrydata/audio/'+ folder + '/'+ chunk_name, format="wav")



for audio_file in raw_audio:
    chop_song(audio_file, raw_audio[audio_file])

In [100]:
import pandas as pd
import librosa 
import numpy as np
'''Chop and Transform each track'''
X = pd.DataFrame(columns = np.arange(45), dtype = 'float32').astype(np.float32)
j = 0
k = 0
l = 0
for i, filename in enumerate(os.listdir('D:/mainproject/babycrydata/audio/pain/')):
    last_number_frames = -1
    if filename.endswith(".wav"):
        #print filename
        audiofile, sr = librosa.load("D:/mainproject/babycrydata/audio/pain/" + filename)
        fingerprint = librosa.feature.mfcc(y=audiofile, sr=sr, n_mfcc=40)
        x = pd.DataFrame(fingerprint, dtype = 'float32')
        x[44] = 'pain'
        X.loc[i] = x.loc[0]
        j = i 
        

for i, filename in enumerate(os.listdir('D:/mainproject/babycrydata/audio/hungry/')):
    if filename.endswith(".wav"):
        #print filename
        audiofile, sr = librosa.load("D:/mainproject/babycrydata/audio/hungry/" + filename)
        fingerprint = librosa.feature.mfcc(y=audiofile, sr=sr, n_mfcc=40)
        x = pd.DataFrame(fingerprint, dtype = 'float32')
        x[44] = 'hungry'
        X.loc[i+j] = x.loc[0] 
        k = i 
        
for i, filename in enumerate(os.listdir('D:/mainproject/babycrydata/audio/discomfort/')):
    if filename.endswith(".wav"):
        #print filename
        audiofile, sr = librosa.load("D:/mainproject/babycrydata/audio/discomfort/" + filename)
        fingerprint = librosa.feature.mfcc(y=audiofile, sr=sr, n_mfcc=40)
        x = pd.DataFrame(fingerprint, dtype = 'float32')
        x[44] = 'discomfort'
        X.loc[i+j+k] = x.loc[0]
        l=i
        
for i, filename in enumerate(os.listdir('D:/mainproject/babycrydata/audio/tired/')):
    if filename.endswith(".wav"):
        #print filename
        audiofile, sr = librosa.load("D:/mainproject/babycrydata/audio/tired/" + filename)
        fingerprint = librosa.feature.mfcc(y=audiofile, sr=sr, n_mfcc=40)
        x = pd.DataFrame(fingerprint, dtype = 'float32')
        x[44] = 'tired'
        X.loc[i+j+k+l] = x.loc[0]

        
#Do something with missing values. you might want to do something more sophisticated with missing values later
X = X.fillna(0)

In [101]:
X.head()

,0,1,2,3,4,5,6,7,8,9,...,35,36,37,38,39,40,41,42,43,44
0,-850.317139,-852.008850,-852.008850,-852.008850,-852.008850,-852.008850,-852.008850,-852.008850,-852.008850,-852.008850,...,-625.902710,-609.206543,-558.989258,-556.160278,-578.153198,-579.574280,-604.941223,-669.555603,-682.124329,pain
1,-709.096252,-702.621033,-680.724060,-665.714844,-643.530151,-597.495728,-583.474304,-582.755676,-593.381714,-624.415344,...,-759.773804,-748.070740,-740.349854,-748.134583,-748.984314,-747.551880,-744.774353,-744.210510,-753.023315,pain
2,-715.216736,-708.238403,-707.021301,-706.583740,-698.491821,-692.686768,-692.701477,-693.114563,-704.256226,-721.227295,...,-703.726501,-679.044128,-653.907776,-609.711121,-560.088623,-535.683777,-515.615356,-496.066223,-466.901672,pain
3,-451.678101,-476.885864,-494.641815,-465.201752,-442.064056,-433.867188,-459.501892,-483.189484,-488.814362,-474.592255,...,-480.836670,-471.485016,-474.537140,-507.281860,-520.139771,-519.343567,-530.151794,-478.971832,-435.418030,pain
4,-434.960358,-431.131592,-432.069855,-446.308655,-475.627777,-480.463257,-466.597382,-458.207245,-462.990845,-455.482727,...,-328.928955,-329.243011,-329.164948,-335.017822,-326.279785,-316.222473,-312.833649,-272.880341,-241.261429,pain


In [102]:
from sklearn.model_selection import train_test_split


y = X[44]
del X[44]
X = X.astype(np.float32)
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [103]:
from IPython.display import HTML
HTML('''<script>
code_show_err=false; 
function code_toggle_err() {
 if (code_show_err){
 $('div.output_stderr').hide();
 } else {
 $('div.output_stderr').show();
 }
 code_show_err = !code_show_err
} 
$( document ).ready(code_toggle_err);
</script>
To toggle on/off output_stderr, click <a href="javascript:code_toggle_err()">here</a>.''')

In [104]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, precision_score, recall_score
forest = RandomForestClassifier()
forest.fit(X_train, y_train)
y_pred_test = forest.predict(X_test)
accuracy_score(y_test, y_pred_test)

0.871859296482412

In [105]:
confusion_matrix(y_test, y_pred_test)

array([[  0,  38,   0,   0],
       [  0, 694,   0,   0],
       [  0,  26,   0,   0],
       [  0,  38,   0,   0]], dtype=int64)

In [111]:
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

  discomfort       0.00      0.00      0.00        38
      hungry       0.87      1.00      0.93       694
        pain       0.00      0.00      0.00        26
       tired       0.00      0.00      0.00        38

    accuracy                           0.87       796
   macro avg       0.22      0.25      0.23       796
weighted avg       0.76      0.87      0.81       796



C:\Users\91902\anaconda3\envs\env\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\91902\anaconda3\envs\env\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\91902\anaconda3\envs\env\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [116]:
#model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [117]:
import pickle

model = RandomForestClassifier()
model.fit(X_train, y_train)
filename = 'D:/mainproject/babycrydata/models/myRandomForest.pkl'
pickle.dump(model, open(filename, 'wb'))

In [123]:
model = pickle.load(open(filename, 'rb'))